## Download & install packages and models

In [1]:
import sys
sys.path.append('..')

In [2]:
from typing import List
import sentence_embedding_evaluation_german as seeg
import torch

/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%%capture
!pip install sentence_transformers

In [4]:
import sentence_transformers as sbert
model_sbert = sbert.SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

## Specify & run sentence embedding evaluation

In [5]:
%%time
# (1) Instantiate your Embedding model
model_sbert = sbert.SentenceTransformer(
    'paraphrase-multilingual-MiniLM-L12-v2')

# (2) Specify the preprocessing
def preprocesser(batch: List[str], params: dict=None) -> List[List[float]]:
    """ Specify your embedding or pretrained encoder here
    Paramters:
    ----------
    params : dict
        The params dictionary
    batch : List[str]
        A list of sentence as string
    Returns:
    --------
    List[List[float]]
        A list of embedding vectors
    """
    features = model_sbert.encode(batch)
    return features

# (3) Training settings
params = {
    'datafolder': '../datasets',
    'bias': True,
    'balanced': True,
    'batch_size': 128, 
    'num_epochs': 500,
    # 'early_stopping': True,
    # 'split_ratio': 0.1,  # if early_stopping=True
    # 'patience': 10,  # if early_stopping=True
}

# (4) Specify downstream tasks
downstream_tasks = ['FCLAIM', 'VMWE', 'OL19-C', 'ABSD-2', 'MIO-P', 'ARCHI', 'LSDC']

# (5) Run experiments
results = seeg.evaluate(downstream_tasks, preprocesser, **params)

CPU times: user 2h 30min 45s, sys: 8min 27s, total: 2h 39min 13s
Wall time: 2h 31min 27s


In [6]:
# save results
import json
with open("seeg-results-sbert.json", 'w') as fp:
    json.dump(results, fp, indent=2)

## Display the results

In [7]:
print("Task | Epochs | N train | N test")
for res in results:
    print(f"{res['task']:>7s}: {res['epochs']:5d} {res['train']['num']:6d} {res['test']['num']:6d}")

Task | Epochs | N train | N test
 FCLAIM:   500   3244    944
   VMWE:   500   6652   1447
 OL19-C:   500   1921    930
 ABSD-2:   500  19432   2555
  MIO-P:   500   4668   4668
  ARCHI:   500  18809   4743
   LSDC:   500  74140   8602


In [8]:
metric = 'acc'  # 'acc', 'acc-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.768  0.708
   VMWE:  0.854  0.830
 OL19-C:  0.789  0.727
 ABSD-2:  0.664  0.633
  MIO-P:  0.900  0.878
  ARCHI:  0.476  0.377
   LSDC:  0.580  0.504


In [9]:
metric = 'acc-balanced'  # 'acc', 'acc-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.765  0.672
   VMWE:  0.826  0.800
 OL19-C:  0.815  0.685
 ABSD-2:  0.705  0.617
  MIO-P:  0.918  0.884
  ARCHI:  0.475  0.377
   LSDC:  0.498  0.418


In [10]:
metric = 'f1'  # 'f1', 'f1-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.768  0.708
   VMWE:  0.854  0.830
 OL19-C:  0.789  0.727
 ABSD-2:  0.664  0.633
  MIO-P:  0.900  0.878
  ARCHI:  0.476  0.377
   LSDC:  0.580  0.504


In [11]:
metric = 'f1-balanced'  # 'f1', 'f1-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.752  0.672
   VMWE:  0.778  0.751
 OL19-C:  0.692  0.611
 ABSD-2:  0.584  0.521
  MIO-P:  0.852  0.820
  ARCHI:  0.474  0.374
   LSDC:  0.367  0.396


In [12]:
# class label distributions (inference)
for res in results:
    print(res['task'], res['test']['num'])
    print(res['test']['distr-test'])
    print(res['test']['distr-pred'])

FCLAIM 944
{'0': '630', '1': '314'}
{'0': '626', '1': '318'}
VMWE 1447
{'1': '256', '0': '1191'}
{'1': '376', '0': '1071'}
OL19-C 930
{'0': '796', '1': '134'}
{'0': '642', '1': '288'}
ABSD-2 2555
{'1': '1670', '0': '780', '2': '105'}
{'0': '983', '1': '1189', '2': '383'}
MIO-P 4668
{'1': '812', '0': '3856'}
{'1': '1208', '0': '3460'}
ARCHI 4743
{'2': '1177', '3': '1176', '1': '1199', '0': '1191'}
{'2': '1442', '3': '1111', '1': '887', '0': '1303'}
LSDC 8602
{'13': '280', '6': '346', '12': '940', '3': '925', '2': '944', '1': '934', '11': '931', '0': '453', '10': '511', '5': '924', '4': '65', '8': '923', '9': '83', '7': '343'}
{'8': '942', '1': '805', '3': '989', '12': '994', '2': '998', '0': '549', '10': '631', '11': '984', '5': '982', '13': '347', '9': '240', '7': '141'}
